<a href="https://colab.research.google.com/github/raqueeb/TensorFlow2/blob/master/embedding_bangla_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### একটা এমবেডিং ট্রেনিং করতে চাচ্ছি

জেফ হিটনের নোটবুক থেকে।

https://github.com/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class_11_03_embedding.ipynb

### বাংলায় বাক্যকে ভেক্টরাইজ করা (ওয়ার্ড২ভেক বা ফাস্টটেক্সট বাদ দিয়ে)
টোকেনাইজ করানোটা দেখাতে চাই 

In [1]:
try:
  # শুধুমাত্র টেন্সর-ফ্লো ২.x ব্যবহার করবো 
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
keras = tf.keras

TensorFlow 2.x selected.


In [0]:
text_non_english = 'আমি আর আসছি না এখানে!'
# tne = unicode(text_non_english, encoding='utf-8')
# print(tne[0])

In [18]:
print(text_non_english[0])

আ


In [20]:
# একটু টেস্ট করি সাইকিট লার্নে 

from sklearn.feature_extraction.text import TfidfVectorizer
corpus = [
     'আমি আর আসছি না এখানে!',
     'অসাধারণ সার্ভিস!',
     'এর থেকে ভালো হয়না',
     'কথা শোনে না ওয়েটার',
      ]
      
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())

['অস', 'আম', 'আর', 'আসছ', 'এখ', 'এর', 'ওয়', 'কথ', 'রণ', 'হয়ন']


In [21]:
print(X.shape)

(4, 10)


In [0]:
from numpy import array
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Embedding, Dense

In [0]:
# Define 10 resturant reviews.
reviews = [
    'আমি আর আসছি না এখানে!',
    'একদম বাজে সার্ভিস',
    'কথা শোনে না ওয়েটার',
    'একদম ঠান্ডা খাবার',
    'বাজে খাবার!',
    'অসাধারণ',
    'অসাধারণ সার্ভিস!',
    'খুব ভালো!',
    'মোটামুটি',
    'এর থেকে ভালো হয়না']

# Define labels (1=negative, 0=positive)
labels = array([1,1,1,1,1,0,0,0,0,0])

In [0]:
# একটু টেস্ট করি সাইকিট লার্নে 

from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer(decode_error='ignore',
                       max_features=4)

X = vect.fit_transform(reviews)

In [0]:
# অথবা টেন্সর-ফ্লো 

from tensorflow.keras.preprocessing.text import Tokenizer

In [0]:
tokenizer = Tokenizer(num_words=20)

# docs = df['quote']
tokenizer.fit_on_texts(reviews)
sequences = tokenizer.texts_to_sequences(reviews)

In [7]:
sequences[:5]

[[8, 9, 10, 1, 11], [2, 3, 4], [12, 13, 1, 14], [2, 15, 5], [3, 5]]

In [8]:
print(reviews[0])

আমি আর আসছি না এখানে!


In [9]:
# ফিরে আসি আগের উদাহরণে 

VOCAB_SIZE = 50
encoded_reviews = [one_hot(d, VOCAB_SIZE) for d in reviews]
print(f"Encoded reviews: {encoded_reviews}")

Encoded reviews: [[23, 6, 33, 17, 25], [6, 10, 45], [36, 43, 17, 13], [6, 28, 16], [10, 16], [29], [29, 45], [47, 1], [25], [9, 5, 1, 46]]


In [10]:
MAX_LENGTH = 4

padded_reviews = pad_sequences(encoded_reviews, maxlen=MAX_LENGTH, padding='post')
print(padded_reviews)

[[ 6 33 17 25]
 [ 6 10 45  0]
 [36 43 17 13]
 [ 6 28 16  0]
 [10 16  0  0]
 [29  0  0  0]
 [29 45  0  0]
 [47  1  0  0]
 [25  0  0  0]
 [ 9  5  1 46]]


In [11]:
model = Sequential()
embedding_layer = Embedding(VOCAB_SIZE, 8, input_length=MAX_LENGTH)
model.add(embedding_layer)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 4, 8)              400       
_________________________________________________________________
flatten (Flatten)            (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 433
Trainable params: 433
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
# fit the model
model.fit(padded_reviews, labels, epochs=100, verbose=0)

In [13]:
print(embedding_layer.get_weights()[0].shape)
print(embedding_layer.get_weights())

(50, 8)
[array([[-3.12827975e-02,  4.86732945e-02,  8.46145451e-02,
        -5.70723526e-02, -1.34908348e-01, -5.42748198e-02,
         1.45759553e-01,  1.47508457e-01],
       [-1.48706838e-01,  1.17172904e-01,  1.29112393e-01,
        -1.00727588e-01,  9.15443078e-02, -9.17942896e-02,
         6.99876472e-02,  1.34916037e-01],
       [-3.20193172e-02,  2.68795751e-02, -3.82953882e-02,
         3.66718657e-02, -7.72655010e-03, -8.14471394e-03,
         2.41601206e-02,  1.09758005e-02],
       [ 4.45945971e-02,  4.41175140e-02, -3.72553244e-02,
         4.93328609e-02,  1.09423324e-03, -3.85251641e-02,
        -2.79844403e-02, -4.79998589e-02],
       [-3.53379846e-02, -2.19841134e-02,  3.79212387e-02,
         4.84202765e-02,  2.16694586e-02, -1.90777425e-02,
         1.50343031e-03,  1.26581267e-03],
       [-1.20961986e-01,  1.07883282e-01,  1.25951886e-01,
         1.08194232e-01,  1.12215348e-01,  8.30505267e-02,
         1.06055588e-01,  7.65256658e-02],
       [-9.78901833e-02, 

In [14]:
loss, accuracy = model.evaluate(padded_reviews, labels, verbose=0)
print(f'Accuracy: {accuracy}')

Accuracy: 1.0
